In [1]:
###
# This is a python notebook that is used to run t-test and calculate the p-values of the fitness scores
# Prepared by: Myra Paz Masinas, Boone Lab, University of Toronto, November 2023
#
# This executes procedure # 72 from the paper: Halder et. al. Design, execution, and analysis of CRISPR–Cas9-based 
# deletions and genetic interaction networks in the fungal pathogen Candida albicans. 
# Nat Protoc 14, 955–975 (2019). https://doi.org/10.1038/s41596-018-0122-6
###

In [2]:
from scipy.stats import ttest_ind as ttest #false_discovery_control
from statsmodels.stats.multitest import fdrcorrection, multipletests
import pandas as pd
import numpy as np

### user-defined variables - update this section as needed

In [3]:
cdn = '15C'  # which condition to process - Change
num_reps = 3  # number of replicates - Change

main_dir = '/Users/violahalder/Desktop/GitHub_GI_Analysis'  # main directory path - Change
eps_path = f'{main_dir}/output/{cdn}_fitness_and_eps.xlsx'  # path to [cdn]_fitness_and_eps.csv file - DO NOT CHANGE

# path to output files
outpath_compiled = f'{main_dir}/output/{cdn}_pvalue_compiled.csv'

### load data per replicate

In [4]:
genes_data = {}
cols = ['Fitness_XY', 'Fitness_YX', 'Fitness_X_Y']
  
data_exp_vs_actual_XY = [] # drop rows with missing/NaN XY values
data_exp_vs_actual_YX = [] # drop rows with missing/NaN YX values
data_actual_XY_vs_YX = [] # drop rows with missing/NaN XY and YX values
for r in range(1, num_reps+1):
    df_rep = pd.read_excel(eps_path, sheet_name=f'R{r}')
    
    # compile fitness scores per gene combination
    for gene_x, gene_y, _, _, _, _, _, _ in df_rep.itertuples(index=False):
        if (gene_x, gene_y) not in genes_data:
            genes_data[(gene_x, gene_y)] = {c: [] for c in cols}
            
        data_gene_combo = df_rep[((df_rep['Gene_X']==gene_x) & (df_rep['Gene_Y']==gene_y))]
        fitness_x = data_gene_combo['Fitness_X'].values[0]
        fitness_y = data_gene_combo['Fitness_Y'].values[0]
        genes_data[(gene_x, gene_y)]['Fitness_X_Y'].append(fitness_x*fitness_y)
        genes_data[(gene_x, gene_y)]['Fitness_XY'].append(data_gene_combo['Fitness_XY'].values[0])
        genes_data[(gene_x, gene_y)]['Fitness_YX'].append(data_gene_combo['Fitness_YX'].values[0])
    
    data_exp_vs_actual_XY.append(df_rep.dropna(subset=['Fitness_XY']).reset_index(drop=True))
    data_exp_vs_actual_YX.append(df_rep.dropna(subset=['Fitness_YX']).reset_index(drop=True))
    data_actual_XY_vs_YX.append(df_rep.dropna(subset=['Fitness_XY', 'Fitness_YX']).reset_index(drop=True))
    print(f'R{r} - all: {df_rep.shape[0]}   exp_vs_XY: {data_exp_vs_actual_XY[r-1].shape[0]}   exp_vs_YX: {data_exp_vs_actual_YX[r-1].shape[0]}   actual_XY_vx_YX: {data_actual_XY_vs_YX[r-1].shape[0]}')

R1 - all: 630   exp_vs_XY: 516   exp_vs_YX: 471   actual_XY_vx_YX: 391
R2 - all: 630   exp_vs_XY: 516   exp_vs_YX: 471   actual_XY_vx_YX: 391
R3 - all: 630   exp_vs_XY: 516   exp_vs_YX: 471   actual_XY_vx_YX: 391


### run t-test: experimental vs actual XY

In [5]:
pval_data = {'Gene_X': [], 'Gene_Y': [], 'P-value (exp_vs_XY)': []}
for gene_x, gene_y, _, _, _, _, _, _ in data_exp_vs_actual_XY[0].itertuples(index=False):
    gene_combo = (gene_x, gene_y)
    values = genes_data[gene_combo]
    A = np.array(values['Fitness_X_Y'])
    B = np.array(values['Fitness_XY'])
    ttest_res = ttest(A, B)
    pval_data['Gene_X'].append(gene_combo[0])
    pval_data['Gene_Y'].append(gene_combo[1])
    pval_data['P-value (exp_vs_XY)'].append(ttest_res.pvalue)
    # print(f'{gene_combo} \n A: {A}\n B: {B} \n p value: {ttest_res.pvalue}') # uncomment for dubugging

# convert to dataframe
df_pval_exp_vs_actual_XY = pd.DataFrame.from_dict(pval_data)

# get p-value adjusted
pvalues = df_pval_exp_vs_actual_XY['P-value (exp_vs_XY)']
pvalues_adjusted_out = fdrcorrection(pvalues, method='indep') # options for method: 'indep' or 'poscorr'
pvalues_adjusted = pvalues_adjusted_out[1]
df_pval_exp_vs_actual_XY['P-value adj (exp_vs_XY)'] = pvalues_adjusted

# print(f'df_pval_exp_vs_actual_XY: {df_pval_exp_vs_actual_XY.shape[0]}') # uncomment for dubugging
df_pval_exp_vs_actual_XY[:1]

,Gene_X,Gene_Y,P-value (exp_vs_XY),P-value adj (exp_vs_XY)
0,TYE7,CAT1,0.584094,0.758274


### run t-test: experimental vs actual YX

In [6]:
pval_data = {'Gene_X': [], 'Gene_Y': [], 'P-value (exp_vs_YX)': []}
for gene_x, gene_y, _, _, _, _, _, _ in data_exp_vs_actual_YX[0].itertuples(index=False):
    gene_combo = (gene_x, gene_y)
    values = genes_data[gene_combo]
    A = np.array(values['Fitness_X_Y'])
    B = np.array(values['Fitness_YX'])
    ttest_res = ttest(A, B)
    pval_data['Gene_X'].append(gene_combo[0])
    pval_data['Gene_Y'].append(gene_combo[1])
    pval_data['P-value (exp_vs_YX)'].append(ttest_res.pvalue)
    #print(f'{gene_combo} \n A: {A}\n B: {B} \n p value: {ttest_res.pvalue}') # Uncomment to debug

# convert to dataframe
df_pval_exp_vs_actual_YX = pd.DataFrame.from_dict(pval_data)

# get p-value adjusted
pvalues = df_pval_exp_vs_actual_YX['P-value (exp_vs_YX)']
pvalues_adjusted_out = fdrcorrection(pvalues, method='indep')
pvalues_adjusted = pvalues_adjusted_out[1]
df_pval_exp_vs_actual_YX['P-value adj (exp_vs_YX)'] = pvalues_adjusted

print(f'df_pval_exp_vs_actual_YX: {df_pval_exp_vs_actual_YX.shape[0]}')
df_pval_exp_vs_actual_YX[:1] # change the number to see more rows of data here

df_pval_exp_vs_actual_YX: 471


,Gene_X,Gene_Y,P-value (exp_vs_YX),P-value adj (exp_vs_YX)
0,TYE7,CAT1,0.788421,0.869664


### run t-test: experimental vs AVG of actual XY and YX

In [7]:
pval_data = {'Gene_X': [], 'Gene_Y': [], 'P-value (exp_vs_average_XY_YX)': []}
for gene_x, gene_y, _, _, _, _, _, _ in df_rep.itertuples(index=False):
    gene_combo = (gene_x, gene_y)
    values = genes_data[gene_combo]
    values_actual_XY = values['Fitness_XY']
    values_actual_YX = values['Fitness_YX']
    
    A = np.array(values['Fitness_X_Y'])
    if np.any(np.isnan(values_actual_XY)) and np.any(np.isnan(values_actual_YX)): 
        continue # skip gene combo
    elif np.any(np.isnan(values_actual_XY)):
        B = np.array(values_actual_YX)
    elif np.any(np.isnan(values_actual_YX)):
        B = np.array(values_actual_XY)
    else:
        B = np.array(values_actual_XY + values_actual_YX)
    ttest_res = ttest(A, B)

    #print(f'{gene_combo} \n A: {A}\n B: {B} \n p value: {ttest_res.pvalue}') # Uncomment to debug
    
    pval_data['Gene_X'].append(gene_combo[0])
    pval_data['Gene_Y'].append(gene_combo[1])
    pval_data['P-value (exp_vs_average_XY_YX)'].append(ttest_res.pvalue)
    #print(f'{gene_combo} — p value: {ttest_res.pvalue}') # Uncomment to debug

# convert to dataframe
df_pval_exp_vs_avg_actual_XY_YX = pd.DataFrame.from_dict(pval_data)

# get p-value adjusted
pvalues = df_pval_exp_vs_avg_actual_XY_YX['P-value (exp_vs_average_XY_YX)']
pvalues_adjusted_out = fdrcorrection(pvalues, method='indep')
pvalues_adjusted = pvalues_adjusted_out[1]
df_pval_exp_vs_avg_actual_XY_YX['P-value adj (exp_vs_average_XY_YX)'] = pvalues_adjusted

print(f'df_pval_exp_vs_avg_actual_XY_YX: {df_pval_exp_vs_avg_actual_XY_YX.shape[0]}')
df_pval_exp_vs_avg_actual_XY_YX[:2] # change the number to see more rows of data here

df_pval_exp_vs_avg_actual_XY_YX: 596


,Gene_X,Gene_Y,P-value (exp_vs_average_XY_YX),P-value adj (exp_vs_average_XY_YX)
0,TYE7,CAT1,0.570173,0.712187
1,YAK1,TYE7,0.556516,0.703339


In [8]:
df_pval_exp_vs_avg_actual_XY_YX.shape

(596, 4)

### check for significant data

In [9]:
# load EPS average data
pval_cutoff = 0.01 # significance cutoff; update the value as needed (0.01 or 0.005)
eps_cutoff_pos = 0.5 # positive interaction eps cutoff; update the value as needed 
eps_cutoff_neg = -0.5 # negative interaction eps cutoff; update the value as needed 

eps_avg_cols = ['Gene_X', 'Gene_Y', 'Average_EPS_XY_EPS_YX']
df_average = pd.read_excel(eps_path, sheet_name='Average', usecols=eps_avg_cols)

significant_data = {'Gene_X': [], 'Gene_Y': [], 'Significant': []}
df_pval_adj = df_pval_exp_vs_avg_actual_XY_YX
for gene_x, gene_y, eps_avg in df_average.itertuples(index=False):
    significant_data['Gene_X'].append(gene_x)
    significant_data['Gene_Y'].append(gene_y)
    
    # get pvalue adjusted, check if below pval_cutoff and then check if EPS is positive or negative
    try:
        pval_adj = df_pval_adj[(df_pval_adj['Gene_X']==gene_x) & 
                               (df_pval_adj['Gene_Y']==gene_y)
                              ]['P-value adj (exp_vs_average_XY_YX)'].values[0]
        if pval_adj < pval_cutoff:
            if eps_avg > eps_cutoff_pos:
                significant_data['Significant'].append(1) # positive interaction
            elif eps_avg < eps_cutoff_neg:
                significant_data['Significant'].append(-1)  # negative interaction
            else:
                significant_data['Significant'].append(0)  # not significant
        else:
            significant_data['Significant'].append(0) # not significant
    except IndexError: 
        significant_data['Significant'].append(np.nan) # missing EPS data

# convert to dataframe
df_significant = pd.DataFrame.from_dict(significant_data)
print(f'df_significant: {df_significant.shape[0]}')
df_significant[:1] # change the number to see more rows of data here

df_significant: 630


,Gene_X,Gene_Y,Significant
0,TYE7,CAT1,0.0


### run t-test: actual_XY vs actual_YX

In [10]:
pval_data = {'Gene_X': [], 'Gene_Y': [], 'P-value (actual_XY_vs_YX)': []}
for gene_x, gene_y, _, _, _, _, _, _ in data_actual_XY_vs_YX[0].itertuples(index=False):
    gene_combo = (gene_x, gene_y)
    values = genes_data[gene_combo]
    A = np.array(values['Fitness_XY'])
    B = np.array(values['Fitness_YX'])
    ttest_res = ttest(A, B)
    pval_data['Gene_X'].append(gene_combo[0])
    pval_data['Gene_Y'].append(gene_combo[1])
    pval_data['P-value (actual_XY_vs_YX)'].append(ttest_res.pvalue)
    #print(f'{gene_combo} — p value: {ttest_res.pvalue}') # Uncomment to debug

# convert to dataframe
df_pval_actual_XY_vs_YX = pd.DataFrame.from_dict(pval_data)

# get p-value adjusted using Benjamini and Hochberg method to correct for false discovery rate
pvalues = df_pval_actual_XY_vs_YX['P-value (actual_XY_vs_YX)']
pvalues_adjusted_out = fdrcorrection(pvalues, method='indep')
pvalues_adjusted = pvalues_adjusted_out[1]
df_pval_actual_XY_vs_YX['P-value adj (actual_XY_vs_YX)'] = pvalues_adjusted

print(f'df_pval_actual_XY_vs_YX: {df_pval_actual_XY_vs_YX.shape[0]}')
df_pval_actual_XY_vs_YX[:1] # change the number to see more rows of data here

df_pval_actual_XY_vs_YX: 391


,Gene_X,Gene_Y,P-value (actual_XY_vs_YX),P-value adj (actual_XY_vs_YX)
0,TYE7,CAT1,0.633691,0.739622


### combine all dataframes

In [11]:
df_pval_all = pd.merge(df_pval_exp_vs_actual_XY, df_pval_exp_vs_actual_YX, how='outer', on=['Gene_X', 'Gene_Y'])
df_pval_all = pd.merge(df_pval_all, df_pval_actual_XY_vs_YX, how='outer', on=['Gene_X', 'Gene_Y'])
df_pval_all = pd.merge(df_pval_all, df_pval_exp_vs_avg_actual_XY_YX, how='outer', on=['Gene_X', 'Gene_Y'])
df_pval_all = pd.merge(df_pval_all, df_significant, how='outer', on=['Gene_X', 'Gene_Y'])
df_pval_all.to_csv(outpath_compiled, index=False)
df_pval_all

,Gene_X,Gene_Y,P-value (exp_vs_XY),P-value adj (exp_vs_XY),P-value (exp_vs_YX),P-value adj (exp_vs_YX),P-value (actual_XY_vs_YX),P-value adj (actual_XY_vs_YX),P-value (exp_vs_average_XY_YX),P-value adj (exp_vs_average_XY_YX),Significant
0,TYE7,CAT1,0.584094,0.758274,0.788421,0.869664,0.633691,0.739622,0.570173,0.712187,0.0
1,YAK1,TYE7,0.861234,0.927760,0.484027,0.638591,0.467941,0.588311,0.556516,0.703339,0.0
2,HSP30,YAK1,0.191150,0.369414,NaN,NaN,NaN,NaN,0.191150,0.412521,0.0
3,HSP30,TYE7,0.169854,0.342362,0.787332,0.869664,0.019029,0.076706,0.267151,0.462855,0.0
4,HSP30,CAT1,0.049594,0.158181,0.118544,0.285463,0.174877,0.315100,0.046421,0.194837,0.0
...,...,...,...,...,...,...,...,...,...,...,...
625,EFG1,YAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,EFG1,CAT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,TRX1,C3_00570C_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,TRX1,C2_10540W_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print("ALL DONE")

ALL DONE
